# **Video Processing Week 3: Analisis Tangan & Aplikasi Interaktif**

Minggu ini kita akan mempelajari **analisis tangan dan aplikasi interaktif** menggunakan teknologi AI melalui MediaPipe. Fokus utama adalah deteksi dan tracking tangan serta jari untuk membangun aplikasi interaktif yang dapat merespons gestur tangan secara real-time.

**Tujuan Pembelajaran:**

Di akhir sesi ini kita akan mampu:
- Menggunakan MediaPipe Hand untuk mendeteksi dan melacak tangan secara real-time
- Mengimplementasikan sistem pengenalan gestur sederhana berdasarkan posisi landmark
- Membangun aplikasi penghitung jari dan deteksi gestur dasar
- Mengembangkan proyek Virtual Painter menggunakan gestur tangan sebagai kontrol

**Topik Praktik:**
- **Hand Landmark Detection**: Deteksi dan tracking 21 titik landmark pada tangan
- **Hand Gesture Recognition**: Pengenalan gestur sederhana seperti menghitung jari dan membedakan kepalan vs telapak terbuka
- **Virtual Painter Project**: Aplikasi interaktif untuk menggambar menggunakan gestur jari telunjuk dan mengubah warna/menghapus dengan gestur telapak terbuka

> *This module is inspired by the development of last semester’s materials.* 


*Sebelum lanjut, kita coba import library yang akan kita butuhkan dulu*

In [1]:
# pip install opencv-contrib-python numpy matplotlib mediapipe ipykernel
# atau
%uv pip install opencv-contrib-python numpy matplotlib mediapipe ipykernel

Note: you may need to restart the kernel to use updated packages.


c:\Users\muham\OneDrive\Desktop\sistem-teknologi-multimedia-122140193\multimedia-uv\Scripts\python.exe: No module named uv


In [2]:
import cv2
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt
import os

## Hand Landmark Detection

kita akan melakukan deteksi 21 titik landmark pada satu atau dua tangan melalui webcam secara real-time

In [3]:
# Inisialisasi MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Konfigurasi Hand Detection
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,  # Deteksi maksimal 2 tangan
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# Buka webcam
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Konversi BGR ke RGB
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb)
    
    # Gambar landmark jika tangan terdeteksi
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # print landmark tangan
            # print(hand_landmarks)
            
            # Gambar landmark dan koneksi
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style()
            )
    
    cv2.imshow("Hand Landmark Detection", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
hands.close()

## Hand Gesture Recognition Sederhana

Kita akan mengimplementasikan pengenalan gestur sederhana dengan menerjemahkan data landmark menjadi informasi gestur seperti menghitung jari yang terangkat dan membedakan antara kepalan tangan dengan telapak terbuka.

### Menerjemahkan data landmark menjadi gestur

Berdasarkan kode sebelumnya, MediaPipe hanya memberi Anda 21 titik data mentah. Ia tidak tahu apa arti "menunjuk" atau "mengepal". Anda harus memberitahunya menggunakan logika Python.

*Lalu bagaimana caranya?* Simpel, dengan cara mengukur jarak atau membandingkan posisi antar titik.

#### **Contoh A: Gestur "Menunjuk" ☝️**

**Logika:** "Sebuah tangan dianggap 'menunjuk' JIKA..."
- Ujung jari telunjuk (Titik 8) lurus dan jauh dari telapak tangan
- **DAN** ujung jari tengah (Titik 12), jari manis (Titik 16), dan kelingking (Titik 20) posisinya "melipat" atau dekat dengan telapak tangan

#### **Contoh B: Gestur "Tangan Mengepal" ✊**

**Logika:** "Sebuah tangan dianggap 'mengepal' JIKA..."
- Semua ujung jari (Titik #8, #12, #16, #20) posisinya dekat dengan telapak tangan
- Jempol (Titik #4) juga dalam posisi melipat

#### Kesimpulan

Dengan membandingkan posisi relatif titik-titik landmark tangan, Anda dapat mengartikan gestur tangan tertentu. Logika ini dapat diperluas untuk mengenali gestur yang lebih kompleks sesuai kebutuhan aplikasi kita.

### Praktik: Menghitung jumlah jari yang terangkat

Untuk menghitung jumlah jari yang terangkat, kita bisa melakukan deteksi ujung dan dasar dari masing-masing jari kemudian membandingkan titiknya secara vertikal:

- jika ujung jari lebih tinggi dari dasar/tengah jarinya, maka jari tersebut terangkat. Secara programatik: `finger_tip.y < finger_base.y`

*Catatan: nilai y=0 dimulai dari atas frame. Artinya, nilai y yang lebih kecil menandakan titik ada di bagian atas dan nilai y yang lebih besar ada di bagian bawah*

In [4]:
def count_fingers(hand_landmarks):
    # Landmark tips dan base untuk setiap jari
    # [TIP_ID, PIP_ID] - PIP adalah sendi tengah jari
    finger_tips = [
        [4, 3],   # Jempol
        [8, 6],   # Telunjuk
        [12, 10], # Tengah
        [16, 14], # Manis
        [20, 18]  # Kelingking
    ]
    
    fingers_up = 0
    
    # Cek setiap jari
    for _, (tip_id, pip_id) in enumerate(finger_tips):
        # Jika tip lebih tinggi (y lebih kecil) dari pip = jari terangkat
        if hand_landmarks.landmark[tip_id].y < hand_landmarks.landmark[pip_id].y:
            fingers_up += 1
    
    return fingers_up


mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
)

# Buka webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Flip frame untuk mirror effect
    frame = cv2.flip(frame, 1)
    h, w = frame.shape[:2]
    
    # Konversi BGR ke RGB
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Deteksi tangan
    results = hands.process(rgb)
    
    # Gambar landmark dan hitung jari
    if results.multi_hand_landmarks and results.multi_handedness:
        for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
            # Gambar landmark tangan
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style()
            )
            
            # Hitung jari yang terangkat
            finger_count = count_fingers(hand_landmarks)
            
            # Tampilkan jumlah jari dan label tangan di bagian atas
            hand_label = handedness.classification[0].label
            cv2.putText(frame, f"{hand_label}: {finger_count} fingers", 
                       (10, 70 + list(zip(results.multi_hand_landmarks, results.multi_handedness)).index((hand_landmarks, handedness)) * 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
    
    # Tampilkan instruksi
    cv2.putText(frame, "Show your fingers to the camera", (10, 30),
               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    cv2.putText(frame, "Press 'q' to quit", (10, h - 20),
               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
    # Tampilkan frame
    cv2.imshow("Finger Counter", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
hands.close()

#### 🔎 Eksplorasi

- Menggunakan logika sederhana yang sudah kita implementasikan, meskipun kita telah mengepalkan tangan kita, program masih membaca ada satu jari yang terangkat. Jari apakah yang menyebabkan hal tersebut? dan ide apa yang kamu punya untuk mengatasi masalah ini?

*Hint: Jempol memiliki orientasi berbeda, mungkin perlu logika terpisah*

### Praktik: Membedakan gestur kepalan tangan VS telapak terbuka

Untuk membedakan gestur kepalan tangan vs telapak terbuka, kita dapat menganalisis posisi landmark jari-jari relatif terhadap telapak tangan. Berikut adalah tahapan implementasinya:

1. Analisis Posisi Jari
Kita membandingkan posisi ujung jari (tip) dengan sendi tengah (PIP) atau pangkal jari:
- **Jari terangkat**: Ujung jari berada di atas sendi tengah (koordinat y lebih kecil)
- **Jari tertutup**: Ujung jari berada di bawah atau sejajar dengan sendi tengah

2. Logika Penentuan Gestur

**Kepalan Tangan (Fist):**
- Semua ujung jari (index 8, 12, 16, 20) berada di bawah sendi tengahnya
- Jempol (index 4) tertutup ke dalam telapak tangan
- Kondisi: `finger_count == 0` atau semua jari dalam posisi tertutup

**Telapak Terbuka (Open Palm):**
- Semua ujung jari berada di atas sendi tengahnya
- Jempol terangkat dan terpisah dari telapak tangan
- Kondisi: `finger_count >= 4` dan jarak antar jari cukup lebar, atau `tip.y < base.y`

In [5]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

def is_open_palm(hand_landmarks):
    fingers_up = 0
    
    # Cek 4 jari (telunjuk, tengah, manis, kelingking)
    finger_tips = [[8, 6], [12, 10], [16, 14], [20, 18]]
    
    for tip_id, pip_id in finger_tips:
        #
        if hand_landmarks.landmark[tip_id].y < hand_landmarks.landmark[pip_id].y:
            fingers_up += 1
    
    return fingers_up >= 3

hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
)

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = cv2.flip(frame, 1)
    h, w = frame.shape[:2]
    
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb)
    
    gesture = False
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            gesture = is_open_palm(hand_landmarks)
            
    color = (0, 255, 0) if gesture else (0, 0, 255)
            
    cv2.putText(frame, f"Status: {'OPEN PALM' if gesture else 'FIST'}", (10, 40),
               cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    cv2.putText(frame, "Press 'q' to quit", (10, h - 20),
               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
    
    cv2.imshow("Fist vs Open Palm", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
hands.close()

## Proyek Mini: Virtual Painter

Kita akan membuat sebuah aplikasi Virtual Painter yang memungkinkan pengguna menggambar di layar menggunakan gestur tangan:

- Menggabungkan deteksi tangan dan gestur
- Gestur jari telunjuk terangkat -> menggambar
- Gestur telapak tangan terbuka -> mengubah warna
- Gestur tangan ditutup -> menghapus gambar

In [6]:
def hand_drawing():
    cap = cv2.VideoCapture(0)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    print(f"Video dimensions: {width}x{height}, FPS: {fps}")
    
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=1,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    )
    mp_drawing = mp.solutions.drawing_utils
    
    # Parameter menggambar
    drawing_color = (0, 0, 255)  # Red color in BGR
    drawing_thickness = 5
    
    # Create a blank canvas for drawing
    canvas = np.zeros((height, width, 3), dtype=np.uint8)
    
    # Previous finger position
    prev_finger_pos = None
    
    # Check if webcam opened successfully
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break
            
        # Balik frame secara horizontal untuk pergerakan natural
        frame = cv2.flip(frame, 1)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)
        
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Draw hand landmarks on the frame
                mp_drawing.draw_landmarks(
                    frame, 
                    hand_landmarks, 
                    mp_hands.HAND_CONNECTIONS
                )
                
                # Dapatkan posisi ujung jari (landmark index 8)
                index_finger_tip = hand_landmarks.landmark[8]
                x = int(index_finger_tip.x * width)
                y = int(index_finger_tip.y * height)
                
                # Gambar titik pada ujung jari
                cv2.circle(frame, (x, y), 10, (0, 255, 0), -1)
                
                # Bandingkan posisi y ujung jari dengan posisi y pangkal jari (landmark index 5)
                index_finger_base = hand_landmarks.landmark[5]
                base_y = int(index_finger_base.y * height)
                
                # Periksa apakah semua jari (telunjuk, tengah, manis, kelingking) terangkat
                # "Terangkat" berarti koordinat y ujung jari lebih kecil dari y pangkal jari
                index_up = y < base_y
                middle_up = hand_landmarks.landmark[12].y < hand_landmarks.landmark[9].y
                ring_up = hand_landmarks.landmark[16].y < hand_landmarks.landmark[13].y
                pinky_up = hand_landmarks.landmark[20].y < hand_landmarks.landmark[17].y
                is_open_palm = index_up and middle_up and ring_up and pinky_up

                if is_open_palm:
                    # Jika telapak tangan terbuka terdeteksi, hapus kanvas
                    canvas = np.zeros((height, width, 3), dtype=np.uint8)
                    prev_finger_pos = None
                elif y < base_y:
                    # jika jari terangkat, gambar pada canvas
                    if prev_finger_pos is not None:
                        cv2.line(canvas, prev_finger_pos, (x, y), drawing_color, drawing_thickness)
                    # perbarui posisi dengan posisi jari sekarang
                    prev_finger_pos = (x, y)
                    
                else:
                    prev_finger_pos = None
        
        # gabungkan frame asli dengan canvas dengan transparansi
        combined_image = cv2.addWeighted(frame, 0.7, canvas, 0.7, 0)
        
        cv2.putText(combined_image, "Raise index finger to draw", (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(combined_image, "Open palm to clear canvas", (10, 60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(combined_image, "Press 'c' to clear canvas", (10, 90), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(combined_image, "Press 'q' to quit", (10, 120), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        cv2.imshow('Hand Drawing App', combined_image)
        
        # Handle keyboard input
        key = cv2.waitKey(1) & 0xFF
        # Press 'q' to quit
        if key == ord('q'):
            break
        elif key == ord('c'):
            canvas = np.zeros((height, width, 3), dtype=np.uint8)
            prev_finger_pos = None
            print("Canvas cleared")
            
    cap.release()
    cv2.destroyAllWindows()

hand_drawing()

Video dimensions: 640x480, FPS: 30
Failed to grab frame


Dari proyek mini ini, ada beberapa topik penting dalam mencapai fungsionalitas yang kita inginkan:

### Menggabungkan deteksi tangan dan gestur

Tahapan:
- Landmark tangan di frame saat ini dideteksi menggunakan mediapipe dengan sintaks `results = hands.process(rgb_frame)`
- Dapatkan posisi dari ujung dan dasar jari telunjuk
- Jika ujung jari lebih tinggi dari dasar jari, maka buat garis dari titik sebelumnya: `cv2.line(canvas, prev_finger_pos, (x, y), drawing_color, drawing_thickness)`
- Simpan posisi jari sekarang sebagai referensi untuk frame berikutnya
- sebaliknya, Jika ujung jari lebih rendah dari dasar jari, hentikan penggambaran garis dan skip frame saat ini `else: prev_finger_pos = None`


### Menggunakan gestur (misal: jari telunjuk terangkat) untuk menggambar di layar.

- Jika ujung jari lebih tinggi dari dasar jari, maka buat garis dari titik sebelumnya: `cv2.line(canvas, prev_finger_pos, (x, y), drawing_color, drawing_thickness)`
- Simpan posisi jari sekarang sebagai referensi untuk frame berikutnya
- sebaliknya, Jika ujung jari lebih rendah dari dasar jari, hentikan penggambaran garis dan skip frame saat ini `else: prev_finger_pos = None`

### Menggunakan gestur lain (telapak terbuka) untuk menghapus kanvas.

Tahapan deteksi gestur telapak terbuka:
- Periksa status semua jari (telunjuk, tengah, manis, kelingking) dengan membandingkan posisi y ujung vs pangkal jari
- Jari dianggap "terangkat" jika koordinat y ujung lebih kecil dari y pangkal: `index_up = y < base_y`
- Untuk jari lainnya: `middle_up = hand_landmarks.landmark[12].y < hand_landmarks.landmark[9].y`
- Jika semua jari terangkat (`is_open_palm = index_up and middle_up and ring_up and pinky_up`), hapus kanvas
- Reset kanvas dengan membuat array kosong: `canvas = np.zeros((height, width, 3), dtype=np.uint8)`
- Reset posisi jari sebelumnya: `prev_finger_pos = None`

## Kesimpulan

Dalam modul **Video Processing Week 3: Analisis Tangan & Aplikasi Interaktif**, kita telah mempelajari implementasi teknologi AI untuk deteksi dan pengenalan gestur tangan menggunakan MediaPipe. Berikut adalah rangkuman pencapaian pembelajaran:

### 🎯 Pencapaian Utama

**1. Hand Landmark Detection**
- Berhasil mengimplementasikan deteksi 21 titik landmark pada tangan secara real-time
- Memahami struktur data landmark MediaPipe dan cara visualisasinya
- Mengonfigurasi parameter deteksi untuk optimalisasi akurasi dan performa

**2. Hand Gesture Recognition**
- Mengembangkan logika penerjemahan data landmark mentah menjadi informasi gestur
- Implementasi penghitung jari dengan membandingkan posisi relatif ujung dan pangkal jari
- Membedakan gestur kepalan tangan vs telapak terbuka menggunakan analisis posisi landmark

**3. Virtual Painter Application**
- Membangun aplikasi interaktif yang merespons gestur tangan secara real-time
- Mengintegrasikan multiple gesture recognition: menggambar (jari telunjuk), menghapus (telapak terbuka)
- Menerapkan teknik overlay canvas untuk visualisasi hasil gambar

### 🔑 Konsep Kunci yang Dipelajari

- **Coordinate System**: Memahami sistem koordinat MediaPipe (nilai y=0 di atas frame)
- **Landmark Analysis**: Teknik membandingkan posisi relatif antar landmark untuk interpretasi gestur
- **Real-time Processing**: Implementasi pipeline deteksi dan response dalam aplikasi interaktif
- **Computer Vision Integration**: Menggabungkan OpenCV dan MediaPipe untuk solusi vision yang kompleks

### 💡 Aplikasi Praktis

Materi ini memberikan foundation yang kuat untuk pengembangan:
- Aplikasi kontrol gestur untuk presentasi atau gaming
- Sistem antarmuka touchless untuk lingkungan steril
- Aplikasi edukasi interaktif untuk anak-anak
- Prototyping human-computer interaction yang inovatif

### 🚀 Pengembangan Selanjutnya

Dengan pemahaman dasar ini, pembelajaran dapat dilanjutkan ke:
- Gesture recognition yang lebih kompleks (sign language, custom gestures)
- Integration dengan machine learning untuk pattern recognition
- Multi-modal interaction (kombinasi hand tracking dengan voice/eye tracking)
- Optimalisasi performa untuk deployment pada edge devices